Primer ejercicio de Spark Structured Streaming. WordCount aplicando filtrado (transformación)

Una vez realizado y entendido este ejemplo te atreverías a aplicar otra transformación, haciendo todas las palabras en minusculas antes de ser contabilizadas, de esta forma las palabras escritas como "Hola", "HOLA", "holA" y "hola", serían contabilizadas como una misma palabra "hola" pista --> pyspark.sql.functions.upper

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split
from pyspark.sql.functions import col

spark = SparkSession.builder.appName("StructuredNetworkWordCount").getOrCreate()

In [2]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
# In command line -> nc -lk 9999
lines =spark.readStream.format("socket").option("host", "localhost").option("port", 9999).load()

In [3]:
# Split the lines into words
words =lines.select(explode(split(lines.value, " ")).alias("word"))

In [4]:
# filtrando los artículos (el, la, los, las, un, una, unos, unas)
filteredWords = words.filter(~col("word").isin(['el', 'la', 'los', 'las', 'un', 'una', 'unos', 'unas']))

In [ ]:
# Generate running word count
wordCounts =filteredWords.groupBy("word").count()

In [ ]:
# Start running the query that prints the running counts to the console
query = wordCounts.writeStream.outputMode("complete").format("console").start()
query.awaitTermination()